# Module 10 Application

## Challenge: Crypto Clustering

In this Challenge, you’ll combine your financial Python programming skills with the new unsupervised learning skills that you acquired in this module.

The CSV file provided for this challenge contains price change data of cryptocurrencies in different periods.

The steps for this challenge are broken out into the following sections:

* Import the Data (provided in the starter code)
* Prepare the Data (provided in the starter code)
* Find the Best Value for `k` Using the Original Data
* Cluster Cryptocurrencies with K-means Using the Original Data
* Optimize Clusters with Principal Component Analysis
* Find the Best Value for `k` Using the PCA Data
* Cluster the Cryptocurrencies with K-means Using the PCA Data
* Visualize and Compare the Results

### Import the Data

This section imports the data into a new DataFrame. It follows these steps:

1. Read  the “crypto_market_data.csv” file from the Resources folder into a DataFrame, and use `index_col="coin_id"` to set the cryptocurrency name as the index. Review the DataFrame.

2. Generate the summary statistics, and use HvPlot to visualize your data to observe what your DataFrame contains.


> **Rewind:** The [Pandas`describe()`function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) generates summary statistics for a DataFrame. 

In [92]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [93]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    Path("Resources/crypto_market_data.csv"),
    index_col="coin_id")

# Display sample data
df_market_data.head(100)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [94]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [95]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

This section prepares the data before running the K-Means algorithm. It follows these steps:

1. Use the `StandardScaler` module from scikit-learn to normalize the CSV file data. This will require you to utilize the `fit_transform` function.

2. Create a DataFrame that contains the scaled data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.


In [96]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled_data = StandardScaler().fit_transform(df_market_data)



In [97]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    scaled_data,
    columns=df_market_data.columns
)

# Copy the crypto names from the original data
df_market_data_scaled["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data

In this section, you will use the elbow method to find the best value for `k`.

1. Code the elbow method algorithm to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following question: What is the best value for `k`?

In [98]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11

k = list(range(1,11))

In [99]:
# Create an empy list to store the inertia values
# YOUR CODE HERE!

inertia = []

In [100]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for i in k:
    k_model = KMeans(n_clusters= i , random_state= 1)
    k_model.fit(df_market_data_scaled)
    inertia.append(k_model.inertia_)



In [101]:
inertia

[287.0,
 195.82021818036043,
 123.19048183836958,
 79.02243535120975,
 65.40592346140596,
 52.933558921015006,
 47.98312409811001,
 37.28818726271726,
 33.06168486647882,
 28.779752431429223]

In [102]:
# Create a dictionary with the data to plot the Elbow curve

elbow_data = {
    'K' : k,
    'inertia' : inertia
}

# Create a DataFrame with the data to plot the Elbow curve
# YOUR CODE HERE!

elbow_data_curve = pd.DataFrame(elbow_data)

In [103]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.

elbow_curve = elbow_data_curve.hvplot(  
    x= "K", 
    y="inertia", 
    title="Elbow Curve",
    xticks=k 
    )

elbow_curve

:Curve   [K]   (inertia)

#### Answer the following question: What is the best value for k?
**Question:** What is the best value for `k`?

**Answer:** # The best value for k will be between 4 and 6. After visualizing the model the best number for K is 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In this section, you will use the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the price changes of cryptocurrencies provided.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the original data.

3. Predict the clusters to group the cryptocurrencies using the original data. View the resulting array of cluster values.

4. Create a copy of the original data and add a new column with the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [104]:
# Initialize the K-Means model using the best value for k

kmeans_model = KMeans(n_clusters= 4, random_state= 1)



In [105]:
# Fit the K-Means model using the scaled data
# YOUR CODE HERE!

kmeans_model.fit(df_market_data_scaled)


KMeans(n_clusters=4, random_state=1)

In [106]:
# Predict the clusters to group the cryptocurrencies using the scaled data
# YOUR CODE HERE!

kmodel_clusters = kmeans_model.predict(df_market_data_scaled)

# View the resulting array of cluster values.


kmodel_clusters

array([3, 3, 1, 1, 3, 3, 3, 3, 3, 1, 1, 1, 1, 3, 1, 3, 1, 1, 3, 1, 1, 3,
       1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 0, 3, 1, 1, 2, 1, 1, 1, 1],
      dtype=int32)

In [107]:
# Create a copy of the DataFrame

kmodel_clusters_predicitons = kmodel_clusters.copy()

In [108]:
# Add a new column to the DataFrame with the predicted clusters

df_market_data_scaled['Cluster'] = kmodel_clusters_predicitons

# Display sample data

df_market_data_scaled

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,Cluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,3
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,3
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,1
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,1
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,3
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,-0.225533,3
chainlink,0.011397,2.572251,1.101647,-0.490495,-0.931954,0.387759,-0.018284,3
cardano,0.102530,1.508001,0.648885,0.328959,-0.486349,0.065080,-0.155428,3
litecoin,0.077497,0.334297,0.858520,-0.012646,-0.366477,-0.486266,-0.292351,3


In [109]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

kmean_scatter_plot = df_market_data_scaled.hvplot.scatter(x="price_change_percentage_24h",
                                     y="price_change_percentage_7d",
                                     by = 'Cluster',
                                     hover_cols ='coin_id'
                                     )

kmean_scatter_plot

:NdOverlay   [Cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis

In this section, you will perform a principal component analysis (PCA) and reduce the features to three principal components.

1. Create a PCA model instance and set `n_components=3`.

2. Use the PCA model to reduce to three principal components. View the first five rows of the DataFrame. 

3. Retrieve the explained variance to determine how much information can be attributed to each principal component.

4. Answer the following question: What is the total explained variance of the three principal components?

5. Create a new DataFrame with the PCA data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.

In [110]:
# Create a PCA model instance and set `n_components=3`.
# YOUR CODE HERE!
 
pca = PCA(n_components= 3)

In [111]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.

market_data_scaled_pca = pca.fit_transform(df_market_data_scaled)

# View the first five rows of the DataFrame. 


market_data_scaled_pca[:5]

array([[-1.58513942,  0.13439444,  0.545621  ],
       [-1.32258066,  0.05449393,  1.11570854],
       [ 0.26633536, -0.53640937, -0.79694588],
       [ 0.25722513, -0.59201286, -0.6465287 ],
       [-2.95224985,  0.42237155,  1.26789353]])

In [112]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.

pca.explained_variance_ratio_

array([0.37005408, 0.32322221, 0.19115222])

In [113]:
pca.explained_variance_ratio_.sum()

0.8844285111826466

#### Answer the following question: What is the total explained variance of the three principal components?

**Question:** What is the total explained variance of the three principal components?

**Answer:** # The total variance of the components is 87.88%, this means that the three principal components collectively explain 88% of the total variance.

In [114]:
# Create a new DataFrame with the PCA data.
# Note: The code for this step is provided for you
# Creating a DataFrame with the PCA data


df_market_data_scaled_pca = pd.DataFrame( market_data_scaled_pca , columns= ['PC1', 'PC2','PC3'])


# Copy the crypto names from the original data


df_market_data_scaled_pca["coin_id"] =df_market_data_scaled.index

# Set the coinid column as index


df_market_data_scaled_pca = df_market_data_scaled_pca.set_index('coin_id')


# Display sample data


df_market_data_scaled_pca.head(15)

,PC1,PC2,PC3
coin_id,,,
bitcoin,-1.585139,0.134394,0.545621
ethereum,-1.322581,0.054494,1.115709
tether,0.266335,-0.536409,-0.796946
ripple,0.257225,-0.592013,-0.646529
bitcoin-cash,-2.952250,0.422372,1.267894
binancecoin,-1.975484,0.528219,0.670145
chainlink,-1.671906,0.178423,2.718598
cardano,-1.539740,0.324517,1.497226
litecoin,-1.347533,-0.132837,0.791063


---

### Find the Best Value for k Using the PCA Data

In this section, you will use the elbow method to find the best value for `k` using the PCA data.

1. Code the elbow method algorithm and use the PCA data to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?

In [115]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11


#I will use the list K that the used at the beguining as it has the same range we are looking for


In [116]:
# Create an empy list to store the inertia values


inertia_pca = [ ]

In [117]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list


for i in k:
    k_model_pca_data = KMeans(n_clusters= i, random_state= 1)
    k_model_pca_data.fit(df_market_data_scaled_pca)
    inertia_pca.append(k_model_pca_data.inertia_)

inertia_pca

[287.00783759231985,
 196.26835760962865,
 111.97202664282966,
 43.205878719552985,
 33.042435121783235,
 24.347640212866626,
 19.77731971115921,
 15.868896599933997,
 12.35105773925925,
 10.16935089732053]

In [118]:
# Create a dictionary with the data to plot the Elbow curve

k_means_pca = {'k' : k , 'inertia' : inertia_pca}

# Create a DataFrame with the data to plot the Elbow curve

k_means_pca_elbow_data = pd.DataFrame(k_means_pca)

k_means_pca_elbow_data

,k,inertia
0,1,287.007838
1,2,196.268358
2,3,111.972027
3,4,43.205879
4,5,33.042435
5,6,24.347640
6,7,19.777320
7,8,15.868897
8,9,12.351058
9,10,10.169351


In [119]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.

pca_elbow_curve = k_means_pca_elbow_data.hvplot(  
    x= "k", 
    y="inertia", 
    title="PCA Elbow Curve",
    xticks=k 
    )


#### Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?
* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, its the same. 

---

### Cluster Cryptocurrencies with K-means Using the PCA Data

In this section, you will use the PCA data and the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the principal components.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the PCA data.

3. Predict the clusters to group the cryptocurrencies using the PCA data. View the resulting array of cluster values.

4. Add a new column to the DataFrame with the PCA data to store the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="PC1"` and `y="PC2"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [120]:
# Initialize the K-Means model using the best value for k

model_pca_values = KMeans(n_clusters= 4, random_state= 1)

In [121]:
# Fit the K-Means model using the PCA data

model_pca_values.fit(df_market_data_scaled_pca)

KMeans(n_clusters=4, random_state=1)

In [122]:
# Predict the clusters to group the cryptocurrencies using the PCA data

model_pca_predict = model_pca_values.predict(df_market_data_scaled_pca)

# View the resulting array of cluster values.

model_pca_predict

array([1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 3, 0, 0, 0, 0],
      dtype=int32)

In [123]:
# Create a copy of the DataFrame with the PCA data

model_pca_predict_cluster = df_market_data_scaled_pca.copy()

# Add a new column to the DataFrame with the predicted clusters

model_pca_predict_cluster["PCA clusters"] = model_pca_predict

# Display sample data

model_pca_predict_cluster.head(10)



,PC1,PC2,PC3,PCA clusters
coin_id,,,,
bitcoin,-1.585139,0.134394,0.545621,1
ethereum,-1.322581,0.054494,1.115709,1
tether,0.266335,-0.536409,-0.796946,0
ripple,0.257225,-0.592013,-0.646529,0
bitcoin-cash,-2.952250,0.422372,1.267894,1
binancecoin,-1.975484,0.528219,0.670145,1
chainlink,-1.671906,0.178423,2.718598,1
cardano,-1.539740,0.324517,1.497226,1
litecoin,-1.347533,-0.132837,0.791063,1


In [124]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
# YOUR CODE HERE!

pca_cluster_scatter_plot = model_pca_predict_cluster.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="PCA clusters",
    title = "Scatter Plot by Stock Segment - PCA=3",
    hover_cols='coin_id')

pca_cluster_scatter_plot


:NdOverlay   [PCA clusters]
   :Scatter   [PC1]   (PC2,coin_id)

---

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

1. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the Elbow Curve that you created to find the best value for `k` with the original and the PCA data.

2. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the cryptocurrencies clusters using the original and the PCA data.

3. Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

> **Rewind:** Back in Lesson 3 of Module 6, you learned how to create composite plots. You can look at that lesson to review how to make these plots; also, you can check [the hvPlot documentation](https://holoviz.org/tutorial/Composing_Plots.html).

In [125]:
# Composite plot to contrast the Elbow curves

pca_elbow_curve + elbow_curve

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [K]   (inertia)

In [126]:
# Compoosite plot to contrast the clusters

pca_cluster_scatter_plot + kmean_scatter_plot

:Layout
   .NdOverlay.I  :NdOverlay   [PCA clusters]
      :Scatter   [PC1]   (PC2,coin_id)
   .NdOverlay.II :NdOverlay   [Cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

#### Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Using fewer features in K-Means clustering simplifies the model and can reduce noise, potentially improving clustering quality. However, it may also result in loss of important information, leading to less meaningful clusters. Therefore, it's essential to carefully select which features to keep or discard.